# 🎯 YOLOv8 Fine-tuning (Object Detection)

> **학습 목표**: YOLOv8로 제조 불량 객체 탐지 모델 학습 및 평가

---

## 📋 학습 내용

1. ✅ YOLOv8 아키텍처 이해 (Anchor-free detection)
2. ✅ YOLO 데이터셋 형식 변환 (COCO → YOLO)
3. ✅ YOLOv8n 모델 Fine-tuning (50 epochs, GPU)
4. ✅ mAP@50, mAP@50-95 평가
5. ✅ Bounding Box 시각화 및 예측
6. ✅ NMS Threshold 튜닝
7. ✅ FPS 벤치마킹 (실시간 추론 성능)

**소요 시간**: 약 60분 (학습 시간 포함)  
**난이도**: ⭐⭐⭐⭐ (고급)  
**사전 지식**: Object Detection 개념, YOLO 아키텍처

---

## 🔧 Step 1: 라이브러리 Import

In [ ]:
# YOLOv8 (Ultralytics)
try:
    from ultralytics import YOLO
    import ultralytics
    print(f"✅ Ultralytics 버전: {ultralytics.__version__}")
except ImportError:
    print("❌ Ultralytics 설치 필요: pip install ultralytics")

# 이미지 처리
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

# 데이터 처리
import pandas as pd
import yaml

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 유틸리티
from pathlib import Path
import shutil
import time
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (14, 6)

print("\n✅ 라이브러리 로드 완료!")

## 📂 Step 2: YOLO 데이터셋 준비

**YOLO Format**:
```
dataset/
├── images/
│   ├── train/
│   └── val/
├── labels/
│   ├── train/
│   └── val/
└── data.yaml
```

In [ ]:
# 데이터셋 경로 설정
dataset_root = Path('../data/defect_detection')
dataset_root.mkdir(exist_ok=True, parents=True)

# YOLO 디렉토리 구조 생성
for split in ['train', 'val', 'test']:
    (dataset_root / 'images' / split).mkdir(exist_ok=True, parents=True)
    (dataset_root / 'labels' / split).mkdir(exist_ok=True, parents=True)

print(f"📂 데이터셋 루트: {dataset_root}")
print(f"\n📁 디렉토리 구조:")
for item in sorted(dataset_root.rglob('*')):
    if item.is_dir():
        depth = len(item.relative_to(dataset_root).parts)
        print(f"{'  ' * depth}└── {item.name}/")

In [ ]:
# 샘플 데이터셋 생성 (실제 데이터가 없는 경우)
# 실제 사용 시 Dataset_XrayInspection.zip 압축 해제 필요

def create_sample_yolo_dataset(dataset_root, num_images=50):
    """샘플 YOLO 데이터셋 생성"""
    print("🔄 샘플 데이터셋 생성 중...")
    
    # 클래스 정의
    classes = ['scratch', 'contamination', 'crack', 'normal']
    
    for split, count in [('train', 35), ('val', 10), ('test', 5)]:
        for idx in range(count):
            # 랜덤 이미지 생성
            img = np.random.randint(0, 256, (640, 640, 3), dtype=np.uint8)
            img_path = dataset_root / 'images' / split / f'img_{idx:04d}.jpg'
            Image.fromarray(img).save(img_path)
            
            # 랜덤 bounding box 생성 (YOLO format: class x_center y_center width height)
            num_boxes = np.random.randint(1, 4)
            label_path = dataset_root / 'labels' / split / f'img_{idx:04d}.txt'
            
            with open(label_path, 'w') as f:
                for _ in range(num_boxes):
                    cls = np.random.randint(0, len(classes) - 1)  # 제외 normal
                    x_center = np.random.uniform(0.2, 0.8)
                    y_center = np.random.uniform(0.2, 0.8)
                    width = np.random.uniform(0.1, 0.3)
                    height = np.random.uniform(0.1, 0.3)
                    f.write(f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    
    print(f"✅ 샘플 데이터셋 생성 완료 (총 {num_images}개 이미지)")
    return classes

# 데이터셋 생성
classes = create_sample_yolo_dataset(dataset_root)

# 데이터셋 통계
for split in ['train', 'val', 'test']:
    img_count = len(list((dataset_root / 'images' / split).glob('*.jpg')))
    label_count = len(list((dataset_root / 'labels' / split).glob('*.txt')))
    print(f"📊 {split:5s}: {img_count}개 이미지, {label_count}개 라벨")

In [ ]:
# data.yaml 생성
data_yaml = {
    'path': str(dataset_root.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': len(classes) - 1,  # 클래스 수 (normal 제외)
    'names': classes[:-1]  # 클래스 이름 (normal 제외)
}

yaml_path = dataset_root / 'data.yaml'
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(data_yaml, f, allow_unicode=True, default_flow_style=False)

print(f"✅ data.yaml 생성: {yaml_path}")
print(f"\n📄 data.yaml 내용:")
print(yaml.dump(data_yaml, allow_unicode=True, default_flow_style=False))

## 🤖 Step 3: YOLOv8 모델 로드

In [ ]:
# YOLOv8 모델 선택 (nano - 가장 빠르고 작음)
model_name = 'yolov8n.pt'  # n (nano), s (small), m (medium), l (large), x (xlarge)

print(f"🔄 YOLOv8 모델 로딩: {model_name}...")

try:
    model = YOLO(model_name)
    
    print(f"\n✅ YOLOv8 모델 로드 완료!")
    print(f"\n📊 모델 정보:")
    print(f"   - 모델 크기: {model_name}")
    print(f"   - Task: {model.task}")
    
    # 모델 아키텍처 요약
    model_info = model.info()
    print(f"\n🏗️ 아키텍처 요약:")
    print(f"   - Layers: {model.model.model[-1].i if hasattr(model.model, 'model') else 'N/A'}")
    print(f"   - Parameters: ~3.0M (YOLOv8n)")
    print(f"   - GFLOPs: ~8.1")
    
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("   ultralytics 설치: pip install ultralytics")

## 🎓 Step 4: Fine-tuning

**학습 설정**:
- Epochs: 50
- Image size: 640
- Batch size: 16 (GPU) / 8 (CPU)
- Optimizer: AdamW

In [ ]:
# 학습 설정
train_params = {
    'data': str(yaml_path),
    'epochs': 50,
    'imgsz': 640,
    'batch': 16,
    'device': 0,  # 0=GPU, 'cpu'=CPU
    'project': '../outputs/yolov8',
    'name': 'defect_detection',
    'patience': 10,  # Early stopping
    'save': True,
    'save_period': 10,  # Save every 10 epochs
    'cache': False,
    'workers': 4,
    'verbose': True
}

print("🎓 학습 설정:")
for key, value in train_params.items():
    print(f"   - {key}: {value}")

print(f"\n💡 학습 예상 시간: ~15-30분 (GPU), ~60-90분 (CPU)")
print(f"   실제 데이터 사용 시 데이터셋 크기에 따라 변동")

In [ ]:
# 모델 학습 실행
print("🔄 YOLOv8 Fine-tuning 시작...\n")

try:
    # 학습 실행
    results = model.train(**train_params)
    
    print("\n✅ 학습 완료!")
    print(f"\n📊 학습 결과:")
    print(f"   - Best epoch: {results.best_epoch if hasattr(results, 'best_epoch') else 'N/A'}")
    print(f"   - 저장 경로: {train_params['project']}/{train_params['name']}")
    
except Exception as e:
    print(f"⚠️ 학습 중 에러: {e}")
    print("   샘플 데이터로 인한 에러일 수 있음")
    print("   실제 데이터셋으로 재시도 권장")

## 📊 Step 5: 모델 평가 (mAP)

**평가 지표**:
- mAP@50: IoU threshold 0.5에서의 평균 정밀도
- mAP@50-95: IoU 0.5~0.95 범위의 평균 정밀도

In [ ]:
# Best 모델 로드
best_model_path = Path(train_params['project']) / train_params['name'] / 'weights' / 'best.pt'

if best_model_path.exists():
    print(f"🔄 Best 모델 로딩: {best_model_path}")
    best_model = YOLO(str(best_model_path))
else:
    print("⚠️ Best 모델을 찾을 수 없음. 기본 모델 사용")
    best_model = model

# 검증 데이터셋 평가
print("\n🔄 모델 평가 중...")

try:
    metrics = best_model.val(data=str(yaml_path), split='val', batch=16)
    
    print("\n✅ 평가 완료!")
    print(f"\n📊 성능 지표:")
    
    # mAP 출력
    if hasattr(metrics, 'box'):
        print(f"   - mAP@50: {metrics.box.map50:.4f}")
        print(f"   - mAP@50-95: {metrics.box.map:.4f}")
        print(f"   - Precision: {metrics.box.p:.4f}")
        print(f"   - Recall: {metrics.box.r:.4f}")
        
        # Per-class mAP
        if hasattr(metrics.box, 'ap_class_index'):
            print(f"\n📊 클래스별 mAP@50:")
            for idx, cls_name in enumerate(classes[:-1]):
                ap50 = metrics.box.ap50[idx] if idx < len(metrics.box.ap50) else 0
                print(f"   - {cls_name}: {ap50:.4f}")
    else:
        print("   평가 지표 추출 실패 (샘플 데이터)")
        
except Exception as e:
    print(f"⚠️ 평가 중 에러: {e}")

## 🎨 Step 6: Bounding Box 시각화

In [ ]:
# 테스트 이미지로 추론
test_images = list((dataset_root / 'images' / 'val').glob('*.jpg'))[:6]

print(f"🔄 추론 실행 중 ({len(test_images)}개 이미지)...")

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, img_path in enumerate(test_images):
    # 추론
    results = best_model(str(img_path), conf=0.25, iou=0.45, verbose=False)
    
    # 결과 이미지 (bounding box 포함)
    result_img = results[0].plot()
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    
    # 시각화
    axes[idx].imshow(result_img)
    
    # Detection 개수
    num_detections = len(results[0].boxes)
    axes[idx].set_title(f'{img_path.stem}\n검출: {num_detections}개', fontsize=10)
    axes[idx].axis('off')

plt.suptitle('YOLOv8 Detection 결과', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n✅ 추론 완료!")

## 🎛️ Step 7: NMS Threshold 튜닝

**NMS (Non-Maximum Suppression)**: 중복 박스 제거

In [ ]:
# 다양한 NMS threshold 테스트
nms_thresholds = [0.3, 0.45, 0.6, 0.75]
conf_threshold = 0.25

test_img = test_images[0]

print(f"🔄 NMS Threshold 튜닝 (confidence={conf_threshold})\n")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

results_summary = []

for idx, iou_thresh in enumerate(nms_thresholds):
    # 추론
    results = best_model(str(test_img), conf=conf_threshold, iou=iou_thresh, verbose=False)
    
    # 결과 이미지
    result_img = results[0].plot()
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    
    # 통계
    num_boxes = len(results[0].boxes)
    avg_conf = results[0].boxes.conf.mean().item() if num_boxes > 0 else 0
    
    results_summary.append({
        'iou_threshold': iou_thresh,
        'num_boxes': num_boxes,
        'avg_confidence': avg_conf
    })
    
    # 시각화
    axes[idx].imshow(result_img)
    axes[idx].set_title(f'IoU={iou_thresh}\nBoxes: {num_boxes}, Avg Conf: {avg_conf:.3f}', fontsize=10)
    axes[idx].axis('off')

plt.suptitle('NMS Threshold 비교', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# 결과 요약
results_df = pd.DataFrame(results_summary)
print("\n📊 NMS Threshold 비교 결과:")
print(results_df.to_string(index=False))

print("\n💡 권장 설정:")
print("   - IoU 0.45: 일반적인 경우 (default)")
print("   - IoU 0.3: 겹치는 객체가 많은 경우")
print("   - IoU 0.6: 중복 제거를 강하게 적용")

## ⚡ Step 8: FPS 벤치마킹

실시간 추론 성능 측정

In [ ]:
# FPS 벤치마킹
print("⚡ FPS 벤치마킹 실행 중...\n")

# 다양한 이미지 크기로 벤치마킹
image_sizes = [320, 640, 1280]
benchmark_results = []

for imgsz in image_sizes:
    print(f"🔄 Image size: {imgsz}×{imgsz}")
    
    # Warmup
    for _ in range(10):
        _ = best_model(str(test_images[0]), imgsz=imgsz, verbose=False)
    
    # 실제 측정
    start_time = time.time()
    num_iterations = 100
    
    for _ in range(num_iterations):
        _ = best_model(str(test_images[0]), imgsz=imgsz, verbose=False)
    
    elapsed_time = time.time() - start_time
    fps = num_iterations / elapsed_time
    latency = (elapsed_time / num_iterations) * 1000  # ms
    
    benchmark_results.append({
        'image_size': imgsz,
        'fps': fps,
        'latency_ms': latency
    })
    
    print(f"   - FPS: {fps:.2f}")
    print(f"   - Latency: {latency:.2f}ms\n")

# 결과 시각화
benchmark_df = pd.DataFrame(benchmark_results)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# FPS
axes[0].bar(benchmark_df['image_size'].astype(str), benchmark_df['fps'], color='skyblue')
axes[0].set_title('FPS by Image Size', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Image Size')
axes[0].set_ylabel('FPS')
axes[0].grid(alpha=0.3)

# Latency
axes[1].bar(benchmark_df['image_size'].astype(str), benchmark_df['latency_ms'], color='coral')
axes[1].set_title('Inference Latency', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Image Size')
axes[1].set_ylabel('Latency (ms)')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 실시간 처리 가능 여부
realtime_fps = 30
realtime_capable = benchmark_df[benchmark_df['fps'] >= realtime_fps]

print("\n📊 벤치마킹 결과:")
print(benchmark_df.to_string(index=False))

print(f"\n💡 실시간 처리 (≥{realtime_fps} FPS):")
if len(realtime_capable) > 0:
    for _, row in realtime_capable.iterrows():
        print(f"   ✅ {row['image_size']}×{row['image_size']}: {row['fps']:.1f} FPS")
else:
    print("   ⚠️ GPU 가속 필요 또는 모델 경량화 권장")

## 💾 Step 9: 결과 저장

In [ ]:
# 출력 디렉토리 생성
output_dir = Path('../outputs')
output_dir.mkdir(exist_ok=True)

# 평가 지표 저장
if hasattr(metrics, 'box'):
    metrics_df = pd.DataFrame({
        'metric': ['mAP@50', 'mAP@50-95', 'Precision', 'Recall'],
        'value': [
            metrics.box.map50,
            metrics.box.map,
            metrics.box.p,
            metrics.box.r
        ]
    })
    metrics_file = output_dir / '03_yolo_metrics.csv'
    metrics_df.to_csv(metrics_file, index=False, encoding='utf-8-sig')
    print(f"✅ 평가 지표 저장: {metrics_file}")

# NMS threshold 결과 저장
nms_file = output_dir / '03_yolo_nms_comparison.csv'
results_df.to_csv(nms_file, index=False, encoding='utf-8-sig')
print(f"✅ NMS 비교 저장: {nms_file}")

# 벤치마킹 결과 저장
benchmark_file = output_dir / '03_yolo_fps_benchmark.csv'
benchmark_df.to_csv(benchmark_file, index=False, encoding='utf-8-sig')
print(f"✅ FPS 벤치마킹 저장: {benchmark_file}")

print("\n🎉 YOLOv8 Fine-tuning 완료!")

---

## 🎯 학습 정리

### ✅ 완료한 내용
1. YOLOv8 아키텍처 이해 및 모델 로드
2. YOLO 데이터셋 형식 변환 (COCO → YOLO txt)
3. YOLOv8n Fine-tuning (50 epochs, GPU 가속)
4. mAP@50, mAP@50-95 평가 지표 분석
5. Bounding Box 시각화 및 예측
6. NMS Threshold 튜닝 (0.3, 0.45, 0.6, 0.75)
7. FPS 벤치마킹 (320, 640, 1280 이미지 크기)

### 💡 핵심 인사이트

- **YOLOv8 vs YOLOv5**:
  - Anchor-free detection (더 간단하고 빠름)
  - Improved backbone (CSPDarknet → C2f)
  - Better accuracy-speed trade-off
  - Ultralytics 통합 API (훈련/추론/배포 일원화)

- **YOLO 데이터 형식**:
  - 각 이미지마다 txt 파일 (class x_center y_center width height)
  - 좌표는 이미지 크기로 정규화 (0~1 범위)
  - data.yaml로 데이터셋 구조 정의

- **mAP 이해**:
  - mAP@50: IoU 0.5 기준 (일반적)
  - mAP@50-95: COCO 메트릭 (더 엄격)
  - Precision vs Recall trade-off 이해 필요

- **NMS Tuning**:
  - IoU 낮을수록 더 많은 박스 보존
  - 겹치는 객체가 많으면 IoU 낮춤 (0.3~0.4)
  - 일반적으로 0.45가 적절

- **실시간 추론**:
  - YOLOv8n: 640×640에서 ~100 FPS (GPU)
  - 320×320: 낮은 해상도, 높은 FPS
  - 1280×1280: 높은 정확도, 낮은 FPS
  - 용도에 따라 이미지 크기 조정

- **실무 활용**:
  - 제조 불량 검출: 표면 결함, 부품 누락
  - 실시간 품질 검사: 컨베이어 벨트 모니터링
  - 안전 관리: 작업자 보호구 착용 감지
  - 재고 관리: 자동 카운팅, 위치 추적

### 📚 다음 단계
- **모델 경량화**: YOLOv8n → TensorRT, ONNX 변환
- **데이터 증강**: Mosaic, MixUp, Augmentation 기법
- **앙상블**: 여러 모델 결합으로 정확도 향상

### 🔗 참고 자료
- [YOLOv8 공식 문서](https://docs.ultralytics.com/)
- [Ultralytics GitHub](https://github.com/ultralytics/ultralytics)
- [YOLO 논문 시리즈](https://arxiv.org/abs/2305.09972)

---

*제조AI 교육 v12 Enhanced | Part 2-2 | 2025.02*